### Prepare the color_text dataset for multimodal color masked model input
- Represent color with bins (bin_range = 16 <16bins> vocabulary max 4096; bin_range = 32 <8bins> vocabulary max 512)
    - Format: color palette for image (max 5 colors)
- Represent text with pre-trained LLM

In [1]:
import pandas as pd
from collections import defaultdict  # For word frequency
import math
import random
import ast
from datetime import datetime
from collections import Counter
import numpy as np
import os

import sys
sys.path.append('../src')

from color_palette_completion.utils.text_emb_creator import save_text_embedding_clip
from color_palette_completion.text_color_model.model_config import Config

representation = Config['representation']
bin_range = Config['bin_range']
max_text_seq_length = {
    'text_contents': Config['Max_Text_Contents_Length'],
    'image_labels': Config['Max_Image_Labels_Length']
}

clusterMode = 'lab_' # training data created by lab color space or rgb
kmeansType = '_sklearn'
langType = '_en'
dataTypes = ['train', 'val', 'test']
textTypes = ['text_contents', 'image_labels']

rawdata_path = '../data/colors'
color_data_path = '../data/t2p/color'
text_data_path = '../data/t2p/text'

text_model = '_clip'
emb_file = 'emb_clip_imagemust_seq'

# Check if directory exists
if not os.path.exists(color_data_path):
    os.makedirs(color_data_path)
    
if not os.path.exists(text_data_path):
    os.makedirs(text_data_path)
    
if not os.path.exists(f'{text_data_path}/{emb_file}'):
    os.makedirs(f'{text_data_path}/{emb_file}')


2023-10-23 02:23:16.562164: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0


In [3]:
def get_color_list_bins(data, column_names):
    color_hist = ''
    for column in column_names:
        if pd.notna(data[column]):
            colors = ast.literal_eval(data[column])
            for color in colors:
                if color_hist != '':
                    color_hist += ' '
                color_hist += f'{math.floor(color[0]/bin_range)}_{math.floor(color[1]/bin_range)}_{math.floor(color[2]/bin_range)}'
    return color_hist

In [4]:
column_names = ['image_colors_lab_reorder', 'svg_colors_lab_reorder', 'text_colors_lab_reorder']

def get_color_metadata(data, representation):

    for column in column_names:
        data[f'{column}'] = data.apply(lambda x: get_color_list_bins(x, [column]), axis=1)
        
    return data

def get_color_hist(data, column_names):
    color_hist = ''
    color_hist += f'{data[column_names[0]]} ; {data[column_names[1]]} ; {data[column_names[2]]}'

    return color_hist

def create_colordata(file_path, representation):
    data = pd.read_csv(file_path)
    data = data.reset_index(drop=True)
    
    metadata = get_color_metadata(data, representation)
    metadata['color_hist'] = metadata.apply(lambda x: get_color_hist(x, column_names), axis=1)
    return metadata

#### Create color corpus and text data: train/val/test

In [5]:
for dataType in dataTypes:
    print(dataType)
    metadata = create_colordata(f'{rawdata_path}/data_colors_labels/crello_labels_palette_{dataType}{kmeansType}_hfreqlabels_imagemust{langType}.csv', representation)
    print(metadata.shape)

    # create color data
    metadata['color_hist'].to_csv(f'{color_data_path}/color_corpus_{representation}_{dataType}{kmeansType}.txt', header=None, index=None, sep=' ')
    # create color vocab from train data
    if dataType == 'train':
        metadata_color_hist = pd.read_csv(f'{color_data_path}/color_corpus_{representation}_{dataType}{kmeansType}.txt', header=None)

        # create sentences
        sentences = [row.split(' ') for row in metadata['color_hist']]
        color_freq = defaultdict(int)
        for sent in sentences:
            for i in sent:
                color_freq[i] += 1
        color_freq.pop(';')
        print(f'color freq size: {len(color_freq)}')
        colors = [a for a in color_freq]
        # colors.remove('\n')
        print(f'color vocab size: {len(colors)}')
        with open(f'{color_data_path}/color_vocab_{representation}_{dataType}{kmeansType}.txt', 'w') as f:
            f.write("[")
            for i in range(len(colors)):
                f.write("'%s'," % colors[i]) if i != len(colors) - 1 else f.write("'%s'" % colors[i])
            f.write("]")

    # create text data
    for textType in textTypes:
        metadata[textType].to_csv(f'{text_data_path}/{textType}_imagemust_{dataType}{langType}.txt', header=None, index=None, sep=' ')


train


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (242,244,246,248,282,284,286,288,290,326,328,330,332,334,336,338,340,342,344,346,348,350,352,382,384,386,388,390,392,394,396,398,400,402,404,406,408,410,412,414,416,418,420,422,424,426,428,430,432,434,436,438,440,442,444,446,448,450,452,454,456,458,460,462,464,466,468,470,472,474,476,478,480,482,484,486,488,490,492,494,496,498,500,502,504,506,508,510,512,514,516,518,520,522,524,526,528,530,532,534,536,538,540,542,544,546,548,550,552,554,556,558,560,562,564,566,568,570,572,574,576,578,580,582,584,586,588,590,592,594,596,598,600,602,604,606,608,610,612,614,616,618,620,622,624,626,628,630,632,634,636,638,640,642,644,646,648,650,652,654,656,658,660,662,664,666,668,670,672,674,676,678,680,682,684,686,688,690,692,694,696,698,700,702,704,706,708,710,712,714,716,718,720,722,724,726,728,730,732,734,736,738,740,742,744,746,748,750,752,754,756,758,760,762,764,766,768,770,772,774,776,778,780,782,784,786,788,790,7

(14020, 1567)
color freq size: 758
color vocab size: 758
val


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (288,290,354,390,392,394,396,398,400,402,404,406,408,410,412,414,416,444,446,448,450,452,454,456,458,460,462,464,466,468,470,472,474,476,478,480,482,484,486,488,490,492,494,496,498,500,502,504,506,508,510,512,514,516,518,520,522,524,526,528,530,532,534,536,538,540,542,544,546,548,550,552,554,556,558,560,562,564,566,568,570,572,574,576,578,580,582,584,586,588,590,592,594,596,598,600,602,716,718,720) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


(1704, 729)
test


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (298,300,302,458,460,462,464,466,468,470,472,474,476,478,480,482,484,486,488,546,548,576,578,580,582,584,586,588,610,612,614,616,618,620,622,624,626,628,630,632,634,636,638,640) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


(1712, 659)


#### Create text embedding for text data

In [22]:
# create text embedding and save
def make_and_parse_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        texts_ = f.readlines()
    for line in texts_:
        yield line

def make_text_data(file_path, dataType, textType):
    text_input = []
    
    text_input_ = make_and_parse_text(file_path)
    for tc in text_input_:
        contents = tc.strip('[]"\n').split(',')

        # separate each phrase as 1 text content
        contents = [c.replace('\'', '').replace('\\n', '. ') for c in contents] # use '.' between sentences of different lines
        text_input.append(contents)
        
    data_path = f"{text_data_path}/{emb_file}"
    print(f'start build {dataType} text contents embedding: {datetime.now().strftime("%Y-%m-%d-%H:%M:%S")}')
    text_contents_emb = save_text_embedding_clip(text_input, data_path, textType, dataType, max_text_seq_length[textType])
    print(f'finish build {dataType} text contents embedding: {datetime.now().strftime("%Y-%m-%d-%H:%M:%S")}')


if __name__ == '__main__':
    for dataType in dataTypes:
        file_path = f'{text_data_path}/text_contents_imagemust_{dataType}{langType}.txt'
        make_text_data(file_path, dataType, 'text_contents') # for seq text embedding building
    for dataType in dataTypes:
        file_path = f'{text_data_path}/image_labels_imagemust_{dataType}{langType}.txt'
        make_text_data(file_path, dataType, 'image_labels') # for seq text embedding building
    

In [14]:
# check the created embedding file
text_input_emb_ = np.loadtxt(f'{text_data_path}/{emb_file}/image_labels_hfreq_emb{text_model}_test{langType}.txt', dtype=float)
text_input_emb_.shape

(17120, 512)